In [1]:
from pptx import Presentation
from bs4 import BeautifulSoup as bs
import markdown
import os
import re

In [2]:
# prs = Presentation("BaseballCard.pptx")
# # bbcard_entity_layout = prs.slide_layouts[7]
# # slide = prs.slides.add_slide(bbcard_entity_layout)
# # title = slide.shapes.title
# # # subtitle = slide.placeholders[1]

# # title.text = "Hello, World!"
# # subtitle.text = "python-pptx was here!"

# prs.save('BlankBBCard.pptx')

### Read in Obsidian files

In [5]:
# case_dir = (input("Enter the name of the Obsidian directory to search: (It must be in this directory)\n")
#            .strip())
# print(case_dir)

case_dir = "Investigation"

In [6]:
case_sub_dirs = [i for i in os.listdir(case_dir) if not "." in i]
case_sub_dirs

# target_dir = (input(f"Choose the subdirectory with entities to create target profiles for:\n{case_sub_dirs}\n")
#              .strip())
# print(target_dir)

target_dir = "People"

In [44]:
entities_to_parse = []

for file in os.listdir(f"{case_dir}/{target_dir}"):
    if file.endswith("md"):
        entities_to_parse.append(file)

print(entities_to_parse)

['Arron Gaines.md', 'Elsie Graves.md', 'Jan Jackson.md', 'Kevin Ellison.md', 'Wilfredo Goodman.md']


In [303]:
entity_data = {}

# Read each markdown file in and convert to HTML then convert to soup
for i in entities_to_parse[0:1]:
    entity_name = i.replace(".md", "")
    entity_data[entity_name] = {}
    
    html = None
    with open(f"{case_dir}/{target_dir}/{i}", "r", encoding="utf-8") as f:
        text = f.read()
    
    html = markdown.markdown(text)
    soup = bs(html)
    entity_data[entity_name]["soup"] = soup

# entity_data

For each person entity, want:
- picture (path to)
- summary details (maybe?)
- associates list
- identifiers list (email, phones, etc.)
- locations list (home, work, etc.)

In [304]:
# Example item using first entity (for testing)
ex = entity_data["Arron Gaines"]
# ex

In [305]:
# # Find picture, get filename, reconstruct path, add to dict

# # Determine picture filename and path
# pic_tag = soup.find("code", text="Picture").next_sibling.string # relies on embedded picture being next
# # print((type(pic_tag)))
# # print(pic_tag)

# pic_filename = re.search("([A-Z\_]{1,}\.[0-9A-Z]{1,7})", pic_tag, re.IGNORECASE)[1]
# # print(pic_filename)

# pic_path = f"Assets/{pic_filename}" # need to check if ext is png, jpg, etc.
# # print(pic_path)

# ex["picture_path"] = pic_path
# ex

In [306]:
# Find all list item tags
all_li_tags = soup.find_all("li")
print(all_li_tags)

[<li><code>Detail</code> Subject is suspected of running a phishing campaign</li>, <li><code>Detail</code> Subject believed to be working with [[Kevin Ellison]]</li>, <li><code>Detail</code> Phishing victims include: [[Elsie Graves]], [[Wilfredo Goodman]], and [[Jan Jackson]]</li>, <li><code>Associate</code> works with: [[Kevin Ellison]]</li>, <li><code>Associate</code> works with other person</li>, <li><code>Associate</code> works with [[Arron Gaines]] as well</li>, <li><code>Identifier</code> (probably uses): [[mglanman@mycbt(.)me]]</li>, <li><code>Identifier</code> uses: [[+1472-217-7096]]</li>, <li><code>Identifier</code> possibly uses: 5712223344</li>, <li><code>Residence Name</code> [[350 Central Park West Apartments]]</li>, <li><code>Residence Address</code> 350 Central Park West, New York, NY 10025{</li>, <li><code>Residence Coordinates</code> 40.79078819123121, -73.96568568759587</li>, <li><code>Work Name</code> undetermined organization at [[30 Rockefeller Plaza]]</li>, <li><

In [307]:
prefixes = {
    "Detail": [
        "Detail",
    ],
    "Associate": [
        "Associate",
    ],
    "Identifier": [
        "Identifier",
    ],
    "Location": [
        "Work Name",
        "Work Address",
        "Work Coordinates",
        "Residence Name",
        "Residence Address",
        "Residence Coordinates",
    ],
}

In [319]:
def gather_li_tags_by_type(x, tag_types):
    code_tag = None
    try:
        code_tag = x.find("code")
    except TypeError:
        print("li did not contain a `Code` tag ")
        return False
    
    if not code_tag:
        return False
    
    if any(tag in code_tag for tag in tag_types):
        return True
    else:
        return False

In [320]:
detail_tags = list(filter(lambda x: gather_li_tags_by_type(x, prefixes["Detail"]), all_li_tags))
detail_tags

[<li><code>Detail</code> Subject is suspected of running a phishing campaign</li>,
 <li><code>Detail</code> Subject believed to be working with [[Kevin Ellison]]</li>,
 <li><code>Detail</code> Phishing victims include: [[Elsie Graves]], [[Wilfredo Goodman]], and [[Jan Jackson]]</li>]

In [321]:
associate_tags = list(filter(lambda x: gather_li_tags_by_type(x, prefixes["Associate"]), all_li_tags))
associate_tags

[<li><code>Associate</code> works with: [[Kevin Ellison]]</li>,
 <li><code>Associate</code> works with other person</li>,
 <li><code>Associate</code> works with [[Arron Gaines]] as well</li>]

In [322]:
identifier_tags = list(filter(lambda x: gather_li_tags_by_type(x, prefixes["Identifier"]), all_li_tags))
identifier_tags

[<li><code>Identifier</code> (probably uses): [[mglanman@mycbt(.)me]]</li>,
 <li><code>Identifier</code> uses: [[+1472-217-7096]]</li>,
 <li><code>Identifier</code> possibly uses: 5712223344</li>]

In [323]:
location_tags = list(filter(lambda x: gather_li_tags_by_type(x, prefixes["Location"]), all_li_tags))
location_tags

[<li><code>Residence Name</code> [[350 Central Park West Apartments]]</li>,
 <li><code>Residence Address</code> 350 Central Park West, New York, NY 10025{</li>,
 <li><code>Residence Coordinates</code> 40.79078819123121, -73.96568568759587</li>,
 <li><code>Work Name</code> undetermined organization at [[30 Rockefeller Plaza]]</li>,
 <li><code>Work Address</code> 30 Rockefeller Plaza, New York, NY 10112</li>,
 <li><code>Work Coordinates</code> 40.758941635700246, -73.97900684718394</li>]

In [326]:
def remove_code_tag_substring(raw_tag):
    cleaned_tag = None
    try:
        cleaned_tag = re.sub("<code>.*</code>", "", str(raw_tag))
        return cleaned_tag
    except:
        print("substring doesn't have a `Code` tag in it")
        return raw_tag

def get_text_inside_li_tags(raw_tag):
    inside_text = None
    try:
        inside_text = re.match("(<li>)(.*)(</li>)", raw_tag)[2]
        return inside_text.strip()
    except:
        return raw_tag

def get_text_from_obsidian_links(raw_tag):
    """Match Obsidian's link format and extract characters between double square brackets"""
    try:
        match_obj = re.match("(.*)(\[\[)(.*)(\]\])(.*)", raw_tag)        
#         print("0", match_obj[0]) # match object
#         print("1", match_obj[1]) # match group 1, etc...
#         print("2", match_obj[2])
#         print("3", match_obj[3])
#         print("4", match_obj[4])
#         print("5", match_obj[5])
        text_before = match_obj[1]
        text_inside = match_obj[3]
        text_after = match_obj[5]
        rebuilt = f"{text_before.strip()} {text_inside.strip()} {text_after.strip()}"
        return rebuilt.strip()
    except TypeError:
        print("did not have any double brackets")
        return raw_tag.strip()

def extract_info_from_li_tag(x):
    cleaned = remove_code_tag_substring(x)
#     print("cleaned1: ", cleaned)
    
    cleaned = get_text_inside_li_tags(cleaned)
#     print("cleaned2: ", cleaned)

    cleaned = get_text_from_obsidian_links(cleaned)
#     print("cleaned3: ", cleaned)

    return cleaned

In [327]:
detail_tags = list(map(lambda x: extract_info_from_li_tag(x), detail_tags))
detail_tags

did not have any double brackets
did not have any double brackets


['Subject is suspected of running a phishing campaign',
 'Subject believed to be working with Kevin Ellison',
 'Phishing victims include: [[Elsie Graves]], Wilfredo Goodman , and Jan Jackson']

In [328]:
associate_tags = list(map(lambda x: extract_info_from_li_tag(x), associate_tags))
associate_tags

did not have any double brackets


['works with: Kevin Ellison',
 'works with other person',
 'works with Arron Gaines as well']

In [329]:
identifier_tags = list(map(lambda x: extract_info_from_li_tag(x), identifier_tags))
identifier_tags

did not have any double brackets


['(probably uses): mglanman@mycbt(.)me',
 'uses: +1472-217-7096',
 'possibly uses: 5712223344']

In [330]:
location_tags = list(map(lambda x: extract_info_from_li_tag(x), location_tags))
location_tags

did not have any double brackets
did not have any double brackets
did not have any double brackets
did not have any double brackets


['350 Central Park West Apartments',
 '350 Central Park West, New York, NY 10025{',
 '40.79078819123121, -73.96568568759587',
 'undetermined organization at 30 Rockefeller Plaza',
 '30 Rockefeller Plaza, New York, NY 10112',
 '40.758941635700246, -73.97900684718394']